In [1]:
import warnings

from tqdm import tqdm

import numpy as np
import pandas as pd
import scipy.optimize
from bokeh.layouts import column
import scipy.special
import scipy.stats as st
import os

import bebi103

import bokeh.io
bokeh.io.output_notebook()

%load_ext blackcellmagic

C:\Users\TB-H\anaconda3\lib\site-packages\dask\dataframe\utils.py:369: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
C:\Users\TB-H\anaconda3\lib\site-packages\dask\dataframe\utils.py:369: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
C:\Users\TB-H\anaconda3\lib\site-packages\dask\dataframe\utils.py:369: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
C:\Users\TB-H\anaconda3\lib\site-packages\bebi103\stan.py:51: UserWarning: Both pystan and cmdstanpy ar

Loading BokehJS ...

INFO:blib2to3.pgen2.driver:Generating grammar tables from C:\Users\TB-H\anaconda3\lib\site-packages\blib2to3\Grammar.txt
INFO:blib2to3.pgen2.driver:Writing grammar tables to C:\Users\TB-H\AppData\Local\black\black\Cache\22.6.0\Grammar3.8.8.final.0.pickle
INFO:blib2to3.pgen2.driver:Writing failed: [Errno 2] No such file or directory: 'C:\\Users\\TB-H\\AppData\\Local\\black\\black\\Cache\\22.6.0\\tmpgamjgww_'
INFO:blib2to3.pgen2.driver:Generating grammar tables from C:\Users\TB-H\anaconda3\lib\site-packages\blib2to3\PatternGrammar.txt
INFO:blib2to3.pgen2.driver:Writing grammar tables to C:\Users\TB-H\AppData\Local\black\black\Cache\22.6.0\PatternGrammar3.8.8.final.0.pickle
INFO:blib2to3.pgen2.driver:Writing failed: [Errno 2] No such file or directory: 'C:\\Users\\TB-H\\AppData\\Local\\black\\black\\Cache\\22.6.0\\tmpcezmrdtm'


# Homework 9.2

Let's start by importing the data.

### a.

In [51]:
# Data path
data_path = "../data"

# Import dataset
df = pd.read_csv(os.path.join(data_path, "caulobacter_growth_events.csv"))

df.head()

in which I will notice how there is a maximum of 20 growth events for bacterium 1 and a total of 87 for bacterium 2.

In [52]:
times1 = [
    df.loc[(df["bacterium"] == 1) & (df["growth event"] == i), "time (min)"].values
    - df.loc[(df["bacterium"] == 1) & (df["growth event"] == i), "time (min)"].values[0]
    + 1
    for i in range(20)
]


areas1 = [
    df.loc[(df["bacterium"] == 1) & (df["growth event"] == i), "area (µm²)"].values
    for i in range(20)
]

times2 = [
    df.loc[(df["bacterium"] == 2) & (df["growth event"] == i), "time (min)"].values
    - df.loc[(df["bacterium"] == 2) & (df["growth event"] == i), "time (min)"].values[0]
    + 1
    for i in range(max(df["growth event"]) + 1)
]


areas2 = [
    df.loc[(df["bacterium"] == 2) & (df["growth event"] == i), "area (µm²)"].values
    for i in range(max(df["growth event"]) + 1)
]

We are working with assuming that we have normally distributed residuals in which we are working with each dataset separately so the following for a single growth event for a single bacterium, indexing i as we traverse along time.

\begin{align}
a_i \sim \text{Norm}(a(t_i;a_0, k), \sigma_i) \;\forall i.
\end{align}
We will start by defining the model in which we have a Normal model with homoscedasticity in which the variances are equal such that we have the following homoscedastic model (in which we are doing so to make the calculations a bit nice):

\begin{align}
a_i \sim \text{Norm}(a(t_i;a_0, k), \sigma) \;\forall i
\end{align}

in which we have that the PDF would be

\begin{align*}
f(a_i ; t_i,a_0, k), \sigma) = \prod_{i=1}^n \frac{1}{\sqrt{2 \pi \sigma^2}} \exp{\left( \frac{-1}{2 \sigma^2} (a_i - a(t_i;a_0, k))^2 \right)}
\end{align*}

in which the likelihood function is
\begin{align*}
L (t_i, a_0, k, \sigma; a_i) = \prod_{i=1}^n \frac{1}{\sqrt{2 \pi \sigma^2}} \exp{\left( \frac{-1}{2 \sigma^2} (a_i - a(t_i;a_0, k))^2 \right)}
\end{align*}
and the log likelihood function is
\begin{align}
\ell(t_i, a_0, k, \sigma; a_i) = -\frac{n}{2}\,\ln 2\pi - \sum_i \ln \sigma_i - \frac{1}{2}\sum_i \left(\frac{a_i - a(t_i;a_0, k)}{\sigma}\right)^2.
\end{align}
We will 
\begin{align}
\ell(t_i, a_0, k, \sigma; a_i) = -\frac{n}{2}\,\ln 2\pi - \sum_i \ln \sigma_i - \frac{1}{2 \sigma^2}\sum_i \left(a_i - a(t_i;a_0, k)\right)^2.
\end{align}

in which we can notice how the sum has no $\sigma$ dependence in which we can focus on minimizing the RSS in which we can apply the same methodology from 8.1 and then notice how  $\sigma^* = \sqrt{\text{RSS}^*/n}$ for its MLE, again following the logic for a Normal homoscedastic model.

We will apply this model for both the linear and exponential equations. Let's look at one of our growth events for the first bacterium to try to make a predictive guess.

In [53]:
# Define the plot
p = bokeh.plotting.figure(
    height=350,
    width=650,
    x_axis_label="time (min)",
    y_axis_label="area (µm²)",
    title="Plot of Bacterium 1 Growth Event 0",
)

p.circle(x=times1[0], y=areas1[0], line_width=2, alpha=0.5, legend_label="Experimental")


bokeh.io.show(p)

To me, this kind of looks linear or exponential?!? I guess we should try to work with both models to find which one would be optimal.

## b

Let's find the MLE parameters for $a_0$ and $k$ using the least squares optimization method for all bacteria and growth events.

In [54]:
def line_area(a0, k, t):
    """Compute area using linear model"""

    return a0 * (1 + k * t)


def line_resid(params, t, a):
    """Residual for linear model."""
    a0, k = params
    return a - line_area(a0, k, t)


def line_area_mle_lstq(data):
    """Compute MLE for parameters in linear area model."""

    tdata = data[0]
    adata = data[1]

    res = scipy.optimize.least_squares(
        line_resid,
        np.array([35, 1]),
        args=(tdata, adata),
        bounds=([0, 0], [np.inf, np.inf]),
    )

    # Compute residual sum of squares from optimal params
    rss_mle = np.sum(line_resid(res.x, tdata, adata) ** 2)

    # Compute MLE for sigma
    sigma_mle = np.sqrt(rss_mle / len(tdata))

    return tuple([x for x in res.x] + [sigma_mle])

Now, let's use it on our data set.

In [55]:
# Define empty list for all eight sets
lin_mles1 = []
lin_mles2 = []

for i in range(20):
    # Extract the times and areas
    time1 = times1[i]
    area1 = areas1[i]

    # Append MLES
    lin_mles1.append(line_area_mle_lstq((time1, area1)))

for i in range(max(df["growth event"])):

    # Extract times and areas
    time2 = times2[i]
    area2 = areas2[i]

    # Append MLEs
    lin_mles2.append(line_area_mle_lstq((time2, area2)))

Now, let's work with setting up the functions necessary to find the MLEs for the exponential model, mirroring the process from the linear least square regression process.

In [56]:
def exp_area(a0, k, t):
    """Compute area using exponential model"""

    return a0 * np.exp(k * t)


def exp_resid(params, t, a):
    """Residual for exp model."""
    a0, k = params
    return a - exp_area(a0, k, t)


def exp_area_mle_lstq(data):
    """Compute MLE for parameters in exponential area model."""

    tdata = data[0]
    adata = data[1]

    res = scipy.optimize.least_squares(
        exp_resid,
        np.array([1, 1]),
        args=(tdata, adata),
        bounds=([0, 0], [np.inf, np.inf]),
    )

    # Compute residual sum of squares from optimal params
    rss_mle = np.sum(exp_resid(res.x, tdata, adata) ** 2)

    # Compute MLE for sigma
    sigma_mle = np.sqrt(rss_mle / len(tdata))

    return tuple([x for x in res.x] + [sigma_mle])

Using our datasets, we find that we get

In [57]:
# Define empty list for all eight sets
exp_mles1 = []
exp_mles2 = []

for i in range(20):
    # Extract the times and areas
    time1 = times1[i]
    area1 = areas1[i]

    # Append MLES
    exp_mles1.append(exp_area_mle_lstq((time1, area1)))

for i in range(max(df["growth event"])):

    # Extract times and areas
    time2 = times2[i]
    area2 = areas2[i]

    # Append MLEs
    exp_mles2.append(exp_area_mle_lstq((time2, area2)))

I want to see how the model looks for a few plots to see how well we fit the data so I am going to look at the first three growth events.

In [58]:
plots = []
for i in range(3):
    p = bokeh.plotting.figure(
        height=250,
        width=550,
        x_axis_label="time (min)",
        y_axis_label="area (µm²)",
        title="Area of Bacterium 1 During Growth Event {}".format(i + 1),
    )

    # Creating plot of data
    p.circle(times1[i], areas1[i], legend_label="Experimental")

    # creating plot using func and MLE
    time_theor = np.linspace(0, max(times1[i]), 1000)
    la0_MLE, lk_MLE = lin_mles1[i][0:2]
    ea0_MLE, ek_MLE = exp_mles1[i][0:2]
    larea_theor = line_area(la0_MLE, lk_MLE, time_theor)
    earea_theor = exp_area(ea0_MLE, ek_MLE, time_theor)
    p.line(
        time_theor,
        larea_theor,
        legend_label="Linear Model",
        line_width=2,
        color="orange",
    )
    p.line(
        time_theor,
        earea_theor,
        legend_label="Exponential Model",
        line_width=2,
        color="purple",
    )

    # plotting params
    p.legend.location = "bottom_right"
    p.legend.click_policy = "hide"
    plots.append(p)

for i in range(3):
    p = bokeh.plotting.figure(
        height=250,
        width=550,
        x_axis_label="time (min)",
        y_axis_label="area (µm²)",
        title="Area of Bacterium 2 During Growth Event {}".format(i + 1),
    )

    # Creating plot of data
    p.circle(times2[i], areas2[i], legend_label="Experimental")

    # creating plot using func and MLE
    time_theor = np.linspace(0, max(times2[i]), 1000)
    la0_MLE, lk_MLE = lin_mles2[i][0:2]
    ea0_MLE, ek_MLE = exp_mles2[i][0:2]
    larea_theor = line_area(la0_MLE, lk_MLE, time_theor)
    earea_theor = exp_area(ea0_MLE, ek_MLE, time_theor)
    p.line(
        time_theor,
        larea_theor,
        legend_label="Linear Model",
        line_width=2,
        color="orange",
    )
    p.line(
        time_theor,
        earea_theor,
        legend_label="Exponential Model",
        line_width=2,
        color="purple",
    )

    # plotting params
    p.legend.location = "bottom_right"
    p.legend.click_policy = "hide"
    plots.append(p)

# show plots
bokeh.plotting.show(column(*plots))

in which for these three plots, it appears that the exponential model fits the pattern of the data more cleanly than the linear case. But I don't think it is wise that I just look at the graphs to make this type of conclusion such that I think a predictive regression is needed.

We have that we will create a predictive regression in which we will do it for both bacterium such that we would need to generate samples containing area vs time for the linear model and for the exponential model in which this would involve the assumption that we are measuring our time exactly and that they determine the area (ignoring variation we model in the residuals) in which we are returning an array of samples that corresponds to one set of measurements for prescribed time in which we have that we need two functions to generate samples for the lines and exponentials.

In [59]:
rg = np.random.default_rng()


def sample_line(a0, k, sigma, t, size=1):
    """Generate samples of area vs time for linear model."""
    samples = np.empty((size, len(t)))

    for i in range(size):
        mu = line_area(a0, k, t)
        samples[i] = np.maximum(0, rg.normal(mu, sigma))

    return samples


def sample_exp(a0, k, sigma, t, size=1):
    """Generate samples of area vs time for linear model."""
    samples = np.empty((size, len(t)))

    for i in range(size):
        mu = exp_area(a0, k, t)
        samples[i] = np.maximum(0, rg.normal(mu, sigma))

    return samples

Now, let's run this function for our both bacterium and for both models.

In [60]:
lin_samples1 = [
    sample_line(*lin_mles1[i], np.linspace(0, max(times1[i]), 1000), size=1000)
    for i in tqdm(range(len(lin_mles1)))
]
lin_samples2 = [
    sample_line(*lin_mles2[i], np.linspace(0, max(times2[i]), 1000), size=1000)
    for i in tqdm(range(len(lin_mles2)))
]
exp_samples1 = [
    sample_exp(*exp_mles1[i], np.linspace(0, max(times1[i]), 1000), size=1000)
    for i in tqdm(range(len(exp_mles1)))
]
exp_samples2 = [
    sample_exp(*exp_mles2[i], np.linspace(0, max(times2[i]), 1000), size=1000)
    for i in tqdm(range(len(exp_mles2)))
]

Now, we can generate a predictive regression model. To prevent the loss of information, we will work with 

In [61]:
p = bokeh.plotting.figure(
    height=1050,
    width=1050,
    x_axis_label="time (min)",
    y_axis_label="area (µm²)",
    title="Predictive Regression Area of Bacterium 1 For Each of the Growth Events for Linear and Exponential Models",
)

for i in range(len(lin_samples1)):
    bebi103.viz.predictive_regression(
        samples=lin_samples1[i] + i,
        samples_x=np.linspace(0, max(times1[i]), 1000),
        data=(times1[i], areas1[i] + i),
        p=p,
    )

for i in range(len(lin_samples1)):
    bebi103.viz.predictive_regression(
        samples=exp_samples1[i] + i,
        samples_x=np.linspace(0, max(times1[i]), 1000) + 200,
        data=(times1[i] + 200, areas1[i] + i),
        p=p,
    )

bokeh.io.show(p)

For this type of plot, we have that the linear situation is on the left while the exponential is on the right. We can notice how we appear to have greater overlap with the exponential cases when looking by eye at the predictive regression although we do have regions for both models in which data points are not within the 95% interval generated. We can especially note for the linear model how the discreptancies appear towards the ends of the line, noticing how for the right end as times increase, we begin to deviate from the linear model. We could note how the linear model could be seen as a first order Taylor approximation of the exponential model in which this would make sense since when doing the exponential model, we are fitting additional components of a series of polynomials. Let's see how this compares to the second bacterium.

In [62]:
p = bokeh.plotting.figure(
    height=1150,
    width=1150,
    x_axis_label="time (min)",
    y_axis_label="area (µm²)",
    title="Area of Bacterium 2 For Each of the Growth Events",
)

for i in range(len(lin_samples2)):
    bebi103.viz.predictive_regression(
        samples=lin_samples2[i] + i,
        samples_x=np.linspace(0, max(times2[i]), 1000),
        data=(times2[i], areas2[i] + i),
        p=p,
    )

for i in range(len(lin_samples2)):
    bebi103.viz.predictive_regression(
        samples=exp_samples2[i] + i,
        samples_x=np.linspace(0, max(times2[i]), 1000) + 200,
        data=(times2[i] + 200, areas2[i] + i),
        p=p,
    )

bokeh.io.show(p)

For this type of plot, we have that the linear situation is on the left while the exponential is on the right. We can notice how this discreptancy is less noticeable as for those that have points that do not overlap with the interval appear for both models. I don't think this is very helpful metric on its own, though when zooming in, we can see how this allows us to zoom and make conclusions for each of the plots in which due to the fact they are 86 of them, I think I am going to focus on evaluating the second bacterium once we do a difference plot for our predictive regression. I can notice that for the most part, it appears that the two models fit the data well though there are still points towards the right end in particular that have deviation for both the linear and exponential model, considering how we are evaluating wether or not we have a linear or exponential model. Let's start by generating the samples, this time by using the actual times from the data as the time parameter.

In [63]:
lin_samples1_d = [
    sample_line(*lin_mles1[i], times1[i], size=10000)
    for i in tqdm(range(len(lin_mles1)))
]
lin_samples2_d = [
    sample_line(*lin_mles2[i], times2[i], size=10000)
    for i in tqdm(range(len(lin_mles2)))
]
exp_samples1_d = [
    sample_exp(*exp_mles1[i], times1[i], size=10000)
    for i in tqdm(range(len(exp_mles1)))
]
exp_samples2_d = [
    sample_exp(*exp_mles2[i], times2[i], size=10000)
    for i in tqdm(range(len(exp_mles2)))
]

We will now display the plot for the first bacterium, combining all the plots together.

In [64]:
plots = []

for i in range(len(lin_samples1)):
    p = bokeh.plotting.figure(
        height=200,
        width=200,
        x_axis_label="time (min)",
        y_axis_label="diff. area (µm²)",
        title="Bact 1 Lin For GE {}".format(i),
    )
    bebi103.viz.predictive_regression(
        samples=lin_samples1_d[i] + i,
        samples_x=times1[i],
        data=(times1[i], areas1[i] + i),
        diff=True,
        p=p,
    )
    plots.append(p)
    p = bokeh.plotting.figure(
        height=200,
        width=200,
        x_axis_label="time (min)",
        y_axis_label="diff. area (µm²)",
        title="Bact 1 Exp For GE {}".format(i),
    )
    bebi103.viz.predictive_regression(
        samples=exp_samples1_d[i] + i,
        samples_x=times1[i],
        data=(times1[i], areas1[i] + i),
        diff=True,
        p=p,
    )
    plots.append(p)


bokeh.io.show(bokeh.layouts.gridplot(plots, ncols=6))

This time we can clearly see the number of poitns that are not within the 95% interval. Unlike earlier, I have a better picture of what is happening now for each of the growth events. We can clearly see cases where there are residuals outside the 95% interval for more points found in exponential than the linear case which is interesting to note like in cases 8 and 2. We can also notice how in general for the middle 68% interval how most of the points from the exponential model tend to have most of its points compared to the linear case. We could note how for the linear scenario, the differences are greatest at the ends which is something I pointed out earlier in which once more we have that as I increase time, our linear model could be potentially seen as a Taylor expansion of an exponential model in which if we have an exponential model, deviating from the smaller times would cause our differences to increase dramatically though we could also note this behavior is present in the beginning as well which we will keep in mind.

In [65]:
plots = []

for i in tqdm(range(len(lin_samples2))):
    p = bokeh.plotting.figure(
        height=200,
        width=200,
        x_axis_label="time (min)",
        y_axis_label="diff. area (µm²)",
        title="Bact 2 Lin For GE {}".format(i),
    )
    bebi103.viz.predictive_regression(
        samples=lin_samples2_d[i] + i,
        samples_x=times2[i],
        data=(times2[i], areas2[i] + i),
        diff=True,
        p=p,
    )
    plots.append(p)
    p = bokeh.plotting.figure(
        height=200,
        width=200,
        x_axis_label="time (min)",
        y_axis_label="diff. area (µm²)",
        title="Bact 2 Exp For GE {}".format(i),
    )
    bebi103.viz.predictive_regression(
        samples=exp_samples2_d[i] + i,
        samples_x=times2[i],
        data=(times2[i], areas2[i] + i),
        diff=True,
        p=p,
    )
    plots.append(p)


bokeh.io.show(bokeh.layouts.gridplot(plots, ncols=6))

in which we can notice how we have a greater variety of results this time around in which it is a bit tricky to do an analysis for all 86 plots. We can thus notice that once more we have some plots that show more poitns for the exponential being outside the 95% interval like events 84 and 85 in which this plot clearly shows that difference compared to our earlier graph. We can also see that for the most part our linear residuals still tend to have the U/parabola shape in which we have the greatest deviations (in the same direction!) towards the early times and the later times.  It is also interesting to note that the pattern for residuals for both tend to show how once more that our 68% interval tends to have more data points for the exponential case compared to the linear case.

### c

To determine which function fits our data well, we will calculate the AIC for it, recalling that the formula is
\begin{align}
\text{AIC} = -2\ell(\theta^*;\text{data}) + 2p,
\end{align}
and that $p$ is the number of free parameters in which we will define the log likelihood for a single bacterium and growth event. 

In [66]:
def line_log_likelihood(params, t, a):
    """Log likelihood of linear model for single bacterium and growth event."""
    a0, k, sigma = params

    mu = line_area(a0, k, t)
    return np.sum(st.norm.logpdf(a, mu, sigma))


def exp_log_likelihood(params, t, a):
    """Log likelihood of exponential model for single bacterium and growth event."""
    a0, k, sigma = params

    mu = exp_area(a0, k, t)
    return np.sum(st.norm.logpdf(a, mu, sigma))

We can now calculate the AIC since we have functions for the log-likelihood, using 1 to denote the linear model and 2 to denote the exponential model.

In [67]:
# Initialize log likelihood
lin_log_like_1 = 0
lin_log_like_2 = 0
exp_log_like_1 = 0
exp_log_like_2 = 0

for i in range(19):
    # Extract the times and areas
    time1 = times1[i]
    area1 = areas1[i]

    # Add to the total log likelihood
    lin_log_like_1 += line_log_likelihood(lin_mles1[i], time1, area1)
    exp_log_like_1 += exp_log_likelihood(exp_mles1[i], time1, area1)


for i in range(max(df["growth event"])):

    # Extract times and areas
    time2 = times2[i]
    area2 = areas2[i]

    # Add to the total log likelihood
    lin_log_like_2 += line_log_likelihood(lin_mles2[i], time2, area2)
    exp_log_like_2 += exp_log_likelihood(exp_mles2[i], time2, area2)

# Combine log likelihood
lin_log_like = lin_log_like_1 + lin_log_like_2
exp_log_like = exp_log_like_1 + exp_log_like_2

# Have three parameters, find AIC
lin_AIC = -2 * (lin_log_like - 321)
exp_AIC = -2 * (exp_log_like - 321)

Let's see the linear AIC

In [22]:
lin_AIC

-38147.872266739156

and then the exponential AIC

In [23]:
exp_AIC

-42893.951308511954

in which we will see how the value of AIC is lower for the exponential case than the linear case in which this implies that the exponential model is strongly preferred which also makes sense when looking at our plot.

We will also calculate the weights in which we get that it is defined as
\begin{align}
w_i = \frac{\mathrm{e}^{-(\text{AIC}_i - \text{AIC}_\mathrm{max})/2}}{\sum_j\mathrm{e}^{-(\text{AIC}_j - \text{AIC}_\mathrm{max})/2}}.
\end{align}
since we have two models it would be
\begin{align}
w_1 = \frac{\mathrm{e}^{-(\text{AIC}_1)/2}}{e^{-(\text{AIC}_1)/2}+e^{-(\text{AIC}_2)/2}} = \frac{1}{1+ e^{-(1/2)(\text{AIC}_2-\text{AIC}_1)}}
\end{align}
and
\begin{align}
w_2 = \frac{\mathrm{e}^{-(\text{AIC}_2)/2}}{e^{-(\text{AIC}_2)/2}+e^{-(\text{AIC}_1)/2}} = \frac{1}{1+ e^{-(1/2)(\text{AIC}_1-\text{AIC}_2)}}
\end{align}

For $w_2$, this would be about 1 due to the difference between the two in the exponential being very large positive number such that it approaches 0 (assigning linear to 1 and exponential to 2). For $w_1$, this would have that the difference would be a very large negative number, yielding a very large denominator and approaching 1 in which if I did the calculation in Python, I would yield an error so I am going to do the limit as I did for both of them. Thus, we can see how we can find that I can rule out the linear model in this scenario and choose to use the exponential model when using the weights.

### Attributions

Tiba and Andrea worked together to solve this problem, both thinking about the conceptual understanding. Tiba coded up the problem, using code from previous lectures. Grace, Nastya, Zack, Kayla and Professor Bois helped the team understand how to approach the problem.

In [24]:
%load_ext watermark
%watermark -v -p pandas,numpy,scipy,os,tqdm,warnings,bebi103,bokeh,black,jupyterlab

Python implementation: CPython
Python version       : 3.8.8
IPython version      : 7.22.0

pandas    : 1.4.4
numpy     : 1.23.3
scipy     : 1.9.3
os        : unknown
tqdm      : 4.59.0
warnings  : unknown
bebi103   : 0.1.13
bokeh     : 2.4.3
black     : 21.12b0
jupyterlab: 3.0.14

